# Python from container

For me it's common task to build interaption with container which handle python program.

So in this section I want to show you how to build such an interaction.

## Python image

You need to create a Python image. The following cell describes the docker file that's used to create the image.

In [1]:
%%writefile python_container/dockerfile
FROM python:3.11.4
WORKDIR program
COPY script.py script.py
COPY requirements.txt requirements.txt
RUN pip3 install -r requirements.txt

Overwriting python_container/dockerfile


The next cell describes the `requirements.txt` we'll need for this container.

In [2]:
%%writefile python_container/requirements.txt
numpy==1.24.2
psycopg2-binary==2.9.6

Overwriting python_container/requirements.txt


And python script that will be used for this example. This script just pushes some data to the postgres database that is accessible in `pg_example_postgres_cont` hostname.

In [3]:
%%writefile python_container/script.py
import psycopg2
import random
import string

# establish connection with database
conn = psycopg2.connect(
    dbname = "docker_app_db",
    user = "docker_app",
    password = "docker_app",
    # interesting that host name match
    # with the container name
    # where database deployed
    host = "pg_example_postgres_cont"
)

# add 20 random values to the database
cur = conn.cursor()
for i in range(20):
    text = ''.join(random.choices(string.ascii_lowercase, k=20))
    query = f"INSERT INTO main_table (id, text) VALUES ('{i}', '{text}');"
    cur.execute(query)
cur.close()

print("Adding records is done!")

# commit changes and close the connection
conn.commit()
conn.close()

Overwriting python_container/script.py


Finally, we build the image.

In [4]:
%%bash
docker build -t pg_example_python python_container &> /dev/null

## Start containers

We need to create at least one table in our postgres database. So in the following cell we store the script that will be used to initialise the database.

In [5]:
%%writefile python_container/create_table.sql
CREATE TABLE main_table(
    id TEXT NOT NULL,
    text TEXT NOT NULL
);

Overwriting python_container/create_table.sql


Next, start the `python` and `postgres` containers.

You should share the same net between containers, in the following example which is completed by `test_project_net`.

In [7]:
%%bash
# network
docker network create test_project_net &> /dev/null
# postgres
docker run --rm -d\
    --name pg_example_postgres_cont\
    -e POSTGRES_USER=docker_app\
    -e POSTGRES_PASSWORD=docker_app\
    -e POSTGRES_DB=docker_app_db\
    --net=test_project_net\
    -v ./python_container/create_table.sql:/docker-entrypoint-initdb.d/create_table.sql\
    postgres:15.4 &> /dev/null
# python
docker run --rm -itd\
    --name pg_example_python_cont\
    --net=test_project_net\
    pg_example_python &> /dev/null

## Executing python

In the next cell, we run the Python script in the Python container. The script simply adds some random values to the `main_table` from the **ohter container**. The message "Adding records is done!" indicates that the Python program was executed normally.

In [10]:
%%bash
docker exec pg_example_python_cont bash -c "python3 script.py"

Traceback (most recent call last):
  File "/program/script.py", line 6, in <module>
    conn = psycopg2.connect(
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/psycopg2/__init__.py", line 122, in connect
    conn = _connect(dsn, connection_factory=connection_factory, **kwasync)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
psycopg2.OperationalError: could not translate host name "pg_example_postgres_cont" to address: Name or service not known



CalledProcessError: Command 'b'docker exec pg_example_python_cont bash -c "python3 script.py"\n'' returned non-zero exit status 1.

## Check the table

So now, to make sure we have done everything right, let us select values from the created table.

In [9]:
%%bash
docker exec pg_example_postgres_cont bash -c \
"psql -U docker_app -d docker_app_db -c \"SELECT * FROM main_table;\""

 id |         text         
----+----------------------
 0  | rqvuxekgkwobqctvgfsz
 1  | zpbrrfefdkfbcunrxnpk
 2  | rxsecyqguuhnxtiddueq
 3  | zeridnjmynyknojmkxqr
 4  | alakdgrejtcrcdoxucke
 5  | ripjvnhaaenmjhrzxhbr
 6  | boletdlslgzhdtdyvzqn
 7  | wiwnxkcxklzhkcfvyyts
 8  | iyauynyiydhjzhzezpbq
 9  | efjzmmsbbruahfkgxsis
 10 | dzcxwyegdgxapfhqpriv
 11 | qimodrjdapolrrueudpy
 12 | xfsgzrcqakqqwuxzmmng
 13 | aisftlhcttaaqkqqdbnu
 14 | yksucssahmtlzlvickif
 15 | afuoyhmdkqvuxervppij
 16 | hczriyvmbsfewdwznqgy
 17 | pziidxiicxqdirbnmcom
 18 | lzlufczgjgluhzzypzzu
 19 | cwiqqnhfcyajqtljdiep
(20 rows)



## Stop containers

In [10]:
%%bash
docker stop pg_example_postgres_cont pg_example_python_cont &> /dev/null
docker network rm test_project_net &> /dev/null

## Clear python image

To avoid creating a lot of rubbish in docker images, you should delete the container created for this example.

In [11]:
%%bash
docker rmi pg_example_python &> /dev/null